In [4]:
!pip install cudf-cu11 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [5]:
!pip install plotly-express

In [8]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2024-11-25 15:17:46--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 13.226.228.31, 13.226.228.97, 13.226.228.119, ...
Connecting to data.rapids.ai (data.rapids.ai)|13.226.228.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474211285 (452M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 452.24M  45.6MB/s    in 5.6s    

2024-11-25 15:17:52 (81.3 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [474211285/474211285]



# Basic Pandas Operations

In [9]:
import pandas as pd

In [10]:
df = pd.read_parquet("nyc_parking_violations_2022.parquet",columns=["Registration State","Violation Description","Vehicle Body Type","Issue Date","Summons Number"])

In [11]:
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
13186831,NY,PHTO SCHOOL ZN SPEED VIOLATION,4DSD,04/29/2022,4777109409
3654218,NY,14-No Standing,SUBN,09/18/2021,8880417939
11655687,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,04/21/2022,4776581279
6067112,NY,PHTO SCHOOL ZN SPEED VIOLATION,DELV,11/26/2021,4758203635
12490217,NY,38-Failure to Dsplay Meter Rec,SUBN,04/11/2022,8758040924
11014712,NH,21-No Parking (street clean),SUBN,03/18/2022,8874178153
2593095,NY,42-Exp Meter (Com Zone),VAN,08/19/2021,8947648164
2650914,NY,38-Failure to Dsplay Meter Rec,VAN,07/31/2021,8953230652
7686740,NJ,19-No Stand (bus stop),DELV,11/30/2021,8914648722
11150598,NY,53-Safety Zone,VAN,03/09/2022,8903683651


In [12]:
%%time
(df[["Registration State","Violation Description"]].value_counts().groupby("Registration State").head(1).sort_index().reset_index())

CPU times: user 2.1 s, sys: 639 ms, total: 2.74 s
Wall time: 2.73 s


,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [13]:
%%time
(df.groupby(["Vehicle Body Type"]).agg({"Summons Number":"count"}).sort_values(["Summons Number"],ascending=False))

CPU times: user 757 ms, sys: 215 ms, total: 973 ms
Wall time: 964 ms


,Summons Number
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
XRL,1
XT,1
YANT,1


# NVIDIA RAPID CODe

In [14]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
!nvidia-smi

Mon Nov 25 15:18:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import cudf

In [4]:
%load_ext cudf.pandas

In [5]:
%%time
import pandas as pd
df1 = pd.read_parquet("nyc_parking_violations_2022.parquet",columns=["Registration State","Violation Description","Vehicle Body Type","Issue Date","Summons Number"])

CPU times: user 506 ms, sys: 225 ms, total: 731 ms
Wall time: 833 ms


In [7]:
df1.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
11969176,NY,20A-No Parking (Non-COM),SUBN,03/31/2022,8867394356
4675286,CT,PHTO SCHOOL ZN SPEED VIOLATION,UT,10/19/2021,4752993089
3866238,NY,38-Failure to Dsplay Meter Rec,VAN,09/09/2021,8950740175
13502567,NY,20-No Parking (Com Plate),VAN,05/26/2022,8584221931
3931021,NY,71A-Insp Sticker Expired (NYS),SUBN,09/04/2021,8965822075
8738892,NY,21-No Parking (street clean),SUBN,01/14/2022,8869236961
14463177,NY,No Parking Street Cleaning,4DSD,06/17/2022,2006467359
6713425,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,12/02/2021,4758982375
9735150,NJ,14-No Standing,VAN,02/16/2022,8868791560
12045134,NY,53-Safety Zone,SUBN,03/29/2022,8893933494


In [8]:
%%time
(df1[["Registration State","Violation Description"]].value_counts().groupby("Registration State").head(1).sort_index().reset_index())

CPU times: user 23 s, sys: 3.11 s, total: 26.1 s
Wall time: 25.9 s


,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [9]:
%%time
(df1.groupby(["Vehicle Body Type"]).agg({"Summons Number":"count"}).sort_values(["Summons Number"],ascending=False))

CPU times: user 22.4 ms, sys: 10 ms, total: 32.4 ms
Wall time: 33.6 ms


,Summons Number
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
YANT,1
YBSD,1
YEL,1
